In [1]:
!pip install pymlask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.11)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.18)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-2

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%cd /content/gdrive/My Drive/Colab Notebooks
# !ls

/content/gdrive/My Drive/Colab Notebooks


In [5]:
import json
import MeCab
from mlask import MLAsk

In [6]:
class TextClassifier:
  def __init__(self, intents_lst):
    self.happy_lst = []
    self.relieved_lst = []
    self.smirking_lst = []
    self.astonished_lst = []
    self.cry_lst = []
    self.angry_lst = []
    self.flushed_lst = []
    self.fearful_lst = []
    self.love_lst = []
    self.squinting_lst = []
    self.boring_lst = []
    self.unpleasant_lst = []
    self.same_lst = []
    self.changed_lst = []
    self.intents_lst = intents_lst

    self.all_count = 0

    self.emotion_analyzer = MLAsk()
    self.mecab = mecab = MeCab.Tagger("-Ochasen")

  def ignore_mlask(self, sentence):  # ML_ASKは単語しか見ていないので、逆説文章に弱い
    if "好きな人" in sentence:  # 今回は失恋ソングが入っていて、「好き」という単語がLove判定に成ってしまうので
      return True
    node = self.mecab.parseToNode(sentence)
    while node:
      # 単語、品詞、詳細情報
      # print(f'{node.surface}\t{node.posid}\t{node.feature}')
      if "助詞,接続助詞" in f'{node.feature}':
        return True
      elif "助詞,格助詞,連語" in f'{node.feature}':
        return True
      elif "記号,読点" in f'{node.feature}':
        return True
      elif "なる" in f'{node.feature}':
        return True
      elif "なれる" in f'{node.feature}':
        return True
      elif "変わる" in f'{node.feature}':
        return True
      elif "変われる" in f'{node.feature}':
        return True
      node = node.next
    return False
  
  """
  def ignore_sentence(self, sentence):  # 名詞だけの文章
    node = self.mecab.parseToNode(sentence)
    found_keiyoushi = False
    fount_not_meishi = False
    count = 0
    while node:
      # 単語、品詞、詳細情報
      # print(f'{node.surface}\t{node.posid}\t{node.feature}')
      if not "名詞" in f'{node.feature}':
        fount_not_meishi = True
      if "形容詞" in f'{node.feature}':
        found_keiyoushi = True
      count += 1
      node = node.next
    if fount_not_meishi:
      if count < 4 and (not found_keiyoushi):
        print("IGNORE: ", sentence)
        return True 
    return False
    """

  def ignore_sentence(self, sentence):  # 名詞だけの文章
    node = self.mecab.parseToNode(sentence)
    while node:
      # 単語、品詞、詳細情報
      # print(f'{node.surface}\t{node.posid}\t{node.feature}')
      if not "名詞" in f'{node.feature}':
        if not "BOS/EOS" in f'{node.feature}':
          return False
      node = node.next
    print("IGNORE: ", sentence)
    return True

  def store_to_lst(self, sentence, intent_name):
    emo_to_intent = {'yorokobi': "Happy", 'yasu': "Relived", 'odoroki': "Astonished", 'aware': "Cry", 'ikari': "Angry",
                     'haji': "Flushed", 'kowa': "Fearful", 'suki': "Love", 'iya': "Unpleasant"}
    res = self.emotion_analyzer.analyze(sentence)
    if 'representative' in res:
      # if len(res['emotion']) > 1:
        # print(res['representative'], res['emotion'], " : ", sentence)
      emotion, text = res['representative']
      if emotion != "takaburi":
        new_intent_name = emo_to_intent[emotion]
        if intent_name != new_intent_name:
          if not self.ignore_mlask(sentence):
            log_text = intent_name + " -> " + new_intent_name + " :" + sentence
            self.changed_lst.append(log_text)
            intent_name = new_intent_name
        else:
          log_text = intent_name + " -> " + new_intent_name + " :" + sentence
          self.same_lst.append(log_text)
    if intent_name == "Happy":
      self.happy_lst.append(sentence)
    elif intent_name == "Relived":
      self.relieved_lst.append(sentence)
    elif intent_name == "Astonished":
      self.astonished_lst.append(sentence)
    elif intent_name == "Cry":
      self.cry_lst.append(sentence)
    elif intent_name == "Angry":
      self.angry_lst.append(sentence)
    elif intent_name == "Flushed":
      self.flushed_lst.append(sentence)
    elif intent_name == "Fearful":
      self.fearful_lst.append(sentence)
    elif intent_name == "Love":
      self.love_lst.append(sentence)
    elif intent_name == "Unpleasant":
      self.unpleasant_lst.append(sentence)
    elif intent_name == "Smirking":
      self.smirking_lst.append(sentence)
    elif intent_name == "Squinting":
      self.squinting_lst.append(sentence)
    elif intent_name == "Boring":
      self.boring_lst.append(sentence)

  def re_classify_text(self, intent_name):
    json_open = open("results/" + intent_name + ".json", 'r')
    json_load = json.load(json_open)
    contents = json_load["userSays"]
    print(intent_name + " : " + str(len(contents)))
    for content in contents:
      sentence = content["data"][0]["text"]
      # if not self.ignore_sentence(sentence):
      self.store_to_lst(sentence, intent_name)
  
  def make_json(self, res_lst, intent_name):
    print(intent_name + " : " + str(len(res_lst)))
    self.all_count += len(res_lst)
    intent_dic =  {"id": "", "name": intent_name, "auto": True, "condition": "",
                    "conditionalFollowupEvents": [], "conditionalResponses": [],
                    "context": [], "contexts": [], "endInteraction": False, "events": [],
                    "fallbackIntent": False, "liveAgentHandoff": False, "parentId": None,
                    "followUpIntents": [], "priority": 500000, "responses": [],
                    "rootParentId": None, "templates": [], "userSays": [],
                    "webhookForSlotFilling": False, "webhookUsed": False}
    responses_dic = {"action": "", "affectedContexts": [], "parameters": [], "defaultResponsePlatforms": {},
                     "messages": [{"type": "message", "condition": "", "speech": []}],
                     "resetContexts": False}
    intent_dic["responses"] = responses_dic
    for sentence in res_lst:
      usersays_dic = {"isTemplate": False, "data": [], "count": 0, "id": "", "updated": None}
      data_dic = {}
      data_dic["text"] = sentence
      data_dic["userDefined"] = False
      usersays_dic["data"].append(data_dic)
      intent_dic["userSays"].append(usersays_dic)

    with open("new_results/" + intent_name + '.json', 'w') as f:
        json.dump(intent_dic, f, indent=4, ensure_ascii=False)
  
  def make_log_json(self, res_lst, save_name):
    print(save_name + " : " + str(len(res_lst)))
    res_dic = {}
    for i in range(len(res_lst)):
      res_dic[i] = res_lst[i]
    with open("new_results/" + save_name + '.json', 'w') as f:
      json.dump(res_dic, f, indent=4, ensure_ascii=False)

  def main(self):
    for intent_name in self.intents_lst:
      self.re_classify_text(intent_name)
    print("===")
    for intent_name in self.intents_lst:
      if intent_name == "Happy":
        self.make_json(self.happy_lst, intent_name)
      elif intent_name == "Relived":
        self.make_json(self.relieved_lst , intent_name)
      elif intent_name == "Astonished":
        self.make_json(self.astonished_lst, intent_name)
      elif intent_name == "Cry":
        self.make_json(self.cry_lst, intent_name)
      elif intent_name == "Angry":
        self.make_json(self.angry_lst, intent_name)
      elif intent_name == "Flushed":
        self.make_json(self.flushed_lst, intent_name)
      elif intent_name == "Fearful":
        self.make_json(self.fearful_lst, intent_name)
      elif intent_name == "Love":
        self.make_json(self.love_lst, intent_name)
      elif intent_name == "Unpleasant":
        self.make_json(self.unpleasant_lst, intent_name)
      elif intent_name == "Smirking":
        self.make_json(self.smirking_lst, intent_name)
      elif intent_name == "Squinting":
        self.make_json(self.squinting_lst, intent_name)
      elif intent_name == "Boring":
        self.make_json(self.boring_lst, intent_name)
    print("===")
    print("All : ", str(self.all_count))
    self.make_log_json(self.changed_lst, "Changed")
    self.make_log_json(self.same_lst, "Same")


In [7]:
INTENTS = ["Happy", "Relived", "Astonished", "Cry", "Angry", "Flushed", "Fearful", "Love", "Unpleasant", "Smirking", "Squinting", "Boring"]
text_classifier = TextClassifier(INTENTS)
text_classifier.main()

Happy : 1839
Relived : 145
Astonished : 41
Cry : 663
Angry : 30
Flushed : 85
Fearful : 68
Love : 678
Unpleasant : 113
Smirking : 23
Squinting : 210
Boring : 14
===
Happy : 1711
Relived : 130
Astonished : 39
Cry : 632
Angry : 30
Flushed : 75
Fearful : 70
Love : 854
Unpleasant : 137
Smirking : 22
Squinting : 197
Boring : 12
===
All :  3909
Changed : 279
Same : 759


In [8]:
# 感情に基づく文章を追加
def add_emotions_to_json(intent_name, add_lst):
  lst = []
  json_open = open("new_results/" + intent_name + ".json", 'r')
  json_load = json.load(json_open)
  contents_dic = json_load["userSays"]
  for sentence in add_lst:
    usersays_dic = {"isTemplate": False, "data": [], "count": 0, "id": "", "updated": None}
    data_dic = {}
    data_dic["text"] = sentence
    data_dic["userDefined"] = False
    usersays_dic["data"].append(data_dic)
    contents_dic.append(usersays_dic)
  json_load["userSays"] = contents_dic
  with open("new_results/" + intent_name + '.json', 'w') as f:
    json.dump(json_load, f, indent=4, ensure_ascii=False)

In [9]:
def make_emotion_lst(emotion_type):

  def ignore_sentence(sentence):  # 名詞だけの文章
    if len(sentence) <= 2:
      return True
    mecab = MeCab.Tagger("-Ochasen")
    node = mecab.parseToNode(sentence)
    while node:
      # 単語、品詞、詳細情報
      # print(f'{node.surface}\t{node.posid}\t{node.feature}')
      if not "名詞" in f'{node.feature}':
        if not "助詞" in f'{node.feature}':
          if not "BOS/EOS" in f'{node.feature}':
            return False
      node = node.next
    # print("IGNORE: ", sentence)
    return True

  lst = []
  f = open("emotions/" + emotion_type + "_uncoded.txt", 'r')
  datalist = f.readlines()
  for data in datalist:
    data = data.replace("\n", "")
    if not ignore_sentence(data):
      lst.append(data)
  print(emotion_type, " : ", len(lst))
  # print(lst)
  return lst

In [10]:
intent_names = ["Happy", "Relived", "Astonished", "Cry", "Angry", "Flushed", "Fearful", "Love", "Unpleasant", "Smirking", "Squinting", "Boring"]
"""
happy_lst = ["うきうきする", "めでたい", "ほくほく"]
relived_lst = ["肩の荷が下りた", "気分が和む", "ほっとする"]
astomished_lst = ["びっくり", "思いもよらない", "どきっとする"]
cry_lst = ["悲しい", "涙ぐましい", "涙ぐましい"]
angry_lst = ["むかつく", "怒る", "頭にきた"]
flushed_lst = ["照れる", "恥ずかしい", "火照る"]
fearful_lst = ["恐ろしい", "ひやひや", "びくびく"]
love_lst = ["愛しい", "惚れ惚れする", "大好き"]
unpleasant_lst = ["気に食わない", "胸が痛くなる", "しょんぼり"]
smirking_lst = []
squinting_lst = []
boring_lst = []
"""

happy_lst = make_emotion_lst('yorokobi')
relived_lst = make_emotion_lst('yasu')
astomished_lst = make_emotion_lst('odoroki')
cry_lst = make_emotion_lst('aware')
angry_lst = make_emotion_lst('ikari')
flushed_lst = make_emotion_lst('haji')
fearful_lst = make_emotion_lst('kowa')
love_lst = make_emotion_lst('suki')
unpleasant_lst = []  # make_emotion_lst('iya')
smirking_lst = []
squinting_lst = []
boring_lst = []

intent_lst_dic = {"Happy": happy_lst, "Relived": relived_lst, "Astonished": astomished_lst, "Cry": cry_lst,
                  "Angry": angry_lst, "Flushed": flushed_lst, "Fearful": fearful_lst, "Love": love_lst,
                  "Unpleasant": unpleasant_lst, "Smirking": smirking_lst, "Squinting": squinting_lst, "Boring": boring_lst}
for intent_name in intent_names:
    add_emotions_to_json(intent_name, intent_lst_dic[intent_name])

yorokobi  :  85
yasu  :  47
odoroki  :  75
aware  :  106
ikari  :  119
haji  :  40
kowa  :  77
suki  :  80


In [11]:
# チェック用
def make_simple_json(intent_name):
    lst = []
    json_open = open("new_results/" + intent_name + ".json", 'r')
    json_load = json.load(json_open)
    contents = json_load["userSays"]
    for content in contents:
      sentence = content["data"][0]["text"]
      lst.append(sentence)
    res_dic = {}
    for i in range(len(lst)):
      res_dic[i] = lst[i]
    with open("check/" + intent_name + '.json', 'w') as f:
      json.dump(res_dic, f, indent=4, ensure_ascii=False)

In [12]:
intent_names = ["Happy", "Relived", "Astonished", "Cry", "Angry", "Flushed", "Fearful", "Love", "Unpleasant", "Smirking", "Squinting", "Boring"]
for intent_name in intent_names:
    make_simple_json(intent_name)

### 以下はデバッグなど

In [ ]:
from mlask import MLAsk
emotion_analyzer = MLAsk()
res = emotion_analyzer.analyze('嬉しい、怒ったよ、悲しいな、怖いよお、恥ずかしい、好きです、嫌いだよ、安心した、びっくり、興奮した')
emotion, text = res['representative']
print(res)
print(emotion, text)

In [ ]:
emotion_analyzer.analyze('辛い時や悲しい時にこの曲聞くと元気になる')

In [ ]:
# res = emotion_analyzer.analyze('おはよう')
# if not 'representative' in res:
#   print("None")
# res = emotion_analyzer.analyze('楽しい')
# if not 'representative' in res:
#   print("None")

In [ ]:
emotion_analyzer.analyze('嫌いだったけどすきになった')

In [33]:
import MeCab
 
mecab = MeCab.Tagger("-Ochasen")
sent = "辛い時や悲しい時にこの曲聞くと元気になる"
res = mecab.parse(sent)
print(res)

辛い	ツライ	辛い	形容詞-自立	形容詞・アウオ段	基本形
時	トキ	時	名詞-非自立-副詞可能		
や	ヤ	や	助詞-並立助詞		
悲しい	カナシイ	悲しい	形容詞-自立	形容詞・イ段	基本形
時	トキ	時	名詞-非自立-副詞可能		
に	ニ	に	助詞-格助詞-一般		
この	コノ	この	連体詞		
曲	キョク	曲	名詞-一般		
聞く	キク	聞く	動詞-自立	五段・カ行イ音便	基本形
と	ト	と	助詞-接続助詞		
元気	ゲンキ	元気	名詞-形容動詞語幹		
に	ニ	に	助詞-格助詞-一般		
なる	ナル	なる	動詞-自立	五段・ラ行	基本形
EOS



In [11]:
import MeCab
 
mecab = MeCab.Tagger("-Ochasen")

In [12]:
sent = "火の棒"
node = mecab.parseToNode(sent)

while node:
    # 単語、品詞、詳細情報をタブ区切りで表示
    print(f'{node.surface}\t{node.posid}\t{node.feature}')
    if "なる" in f'{node.feature}':
      print("!! Found")
    elif "接続助詞" in f'{node.feature}':
      print("!!found")
    # 次の要素を取得
    node = node.next

	0	BOS/EOS,*,*,*,*,*,*,*,*
	38	名詞,一般,*,*,*,*,火,ヒ,ヒ
	24	助詞,連体化,*,*,*,*,の,ノ,ノ
	38	名詞,一般,*,*,*,*,棒,ボウ,ボー
	0	BOS/EOS,*,*,*,*,*,*,*,*


In [35]:
sent = "辛かったけど、この曲聞いて元気になった"
node = mecab.parseToNode(sent)

while node:
    # 単語、品詞、詳細情報をタブ区切りで表示
    print(f'{node.surface}\t{node.posid}\t{node.feature}')
    if "なる" in f'{node.feature}':
      print("!! Found")
    # 次の要素を取得
    node = node.next

	0	BOS/EOS,*,*,*,*,*,*,*,*
辛かっ	10	形容詞,自立,*,*,形容詞・アウオ段,連用タ接続,辛い,ツラカッ,ツラカッ
た	25	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
けど	18	助詞,接続助詞,*,*,*,*,けど,ケド,ケド
、	9	記号,読点,*,*,*,*,、,、,、
この	68	連体詞,*,*,*,*,*,この,コノ,コノ
曲	38	名詞,一般,*,*,*,*,曲,キョク,キョク
聞い	31	動詞,自立,*,*,五段・カ行イ音便,連用タ接続,聞く,キイ,キイ
て	18	助詞,接続助詞,*,*,*,*,て,テ,テ
元気	40	名詞,形容動詞語幹,*,*,*,*,元気,ゲンキ,ゲンキ
に	13	助詞,格助詞,一般,*,*,*,に,ニ,ニ
なっ	31	動詞,自立,*,*,五段・ラ行,連用タ接続,なる,ナッ,ナッ
!! Found
た	25	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
	0	BOS/EOS,*,*,*,*,*,*,*,*


In [18]:
sent = "快"
node = mecab.parseToNode(sent)

while node:
    # 単語、品詞、詳細情報をタブ区切りで表示
    print(f'{node.surface}\t{node.posid}\t{node.feature}')
    if "なる" in f'{node.feature}':
      print("!! Found")
    # 次の要素を取得
    node = node.next

	0	BOS/EOS,*,*,*,*,*,*,*,*
快	10	形容詞,自立,*,*,形容詞・アウオ段,ガル接続,快い,ココロヨ,ココロヨ
	0	BOS/EOS,*,*,*,*,*,*,*,*
